In [1]:
import os
import re
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import boto3
import pyrootutils

pyrootutils.setup_root(os.curdir, indicator=".project-root", pythonpath=True)
from extras import constants, paths
from aws import rds, s3
from dailyjou.utils import search, utils

In [2]:
search.get_product_li(constants.DAILYJOU_TOP_PAGE_URL + "&page=2")

[<li class="xans-record-" id="anchorBoxId_15317">
 <div class="thumbnail">
 <div class="prdImg">
 <a href="/product/세릴-딥-라운드-반팔-티셔츠-3-color/15317/category/46/display/1/" name="anchorBoxName_15317"><img alt="세릴 딥 라운드 반팔 티셔츠 - 3 color" id="eListPrdImage15317_1" src="//dailyjou.com/web/product/medium/202309/4ff5850a50ca2dea19101717be2fcd48.jpg"/></a>
 </div>
 <div class="icon">
 <div class="wish"></div>
 <div class="likeButton displaynone"><button type="button"><strong></strong></button></div>
 <div class="option"><a "="" onclick="optionPreview(this,'listnormal-0',15317,'T')"><img alt="" class="option_preview" id="btn_preview_listnormal-015317" src="//img.echosting.cafe24.com/design/skin/default/product/btn_option_view.gif"/></a> </div>
 </div>
 </div>
 <div class="description">
 <strong class="name"><a class="" href="/product/세릴-딥-라운드-반팔-티셔츠-3-color/15317/category/46/display/1/"><span class="title displaynone"><span style="font-size:12px;color:#555555;">상품명</span> :</span> <span style="f

## 티셔츠 크롤링

### 썸네일 이미지 & 상품 url

In [2]:
page_url = "https://dailyjou.com/product/list.html?cate_no=46&page=2"

In [3]:
response = requests.get(page_url, headers={"User-Agent": "Mozilla/5.0"})
html = response.text
soup = BeautifulSoup(html, "html.parser")

In [4]:
products_li = soup.select('ul.prdList2.grid4 > [id^="anchorBoxId_"]')

In [5]:
product_li = products_li[3]

In [6]:
product_url = "https://dailyjou.com" + product_li.select_one("div.thumbnail > div.prdImg > a")["href"]
product_url

'https://dailyjou.com/product/베베-빈티지-레터링-오버핏-후드-티셔츠-3-color/15355/category/46/display/1/'

In [7]:
thumbnail_img_url = "https:" + product_li.select_one("div.thumbnail > div.prdImg > a > img")["src"]
thumbnail_img_url

'https://dailyjou.com/web/product/medium/202309/2e8c0daa550ac5c26e6ccbd3e0c58821.gif'

### 상품 상세 정보

In [19]:
product_url = "https://dailyjou.com/product/%ED%82%A4%EB%A5%B4%ED%86%A0-%EB%82%98%EC%8B%9C-%EC%85%94%EB%A7%81-%EC%98%A4%ED%94%84%EC%88%84%EB%8D%94-%ED%8B%B0%EC%85%94%EC%B8%A0-%EC%84%B8%ED%8A%B8-3color/15458/category/46/display/1/"

In [20]:
response = requests.get(product_url, headers={"User-Agent": "Mozilla/5.0"})
html = response.text
soup = BeautifulSoup(html, "html.parser")
product_dict = {}

#### product disabled

In [9]:
product_dict['disabled'] = "FALSE"

#### product price

In [10]:
try:
    product_dict['price'] = int(re.sub(r'\([^)]*\)|,', '', soup.select_one("span#span_product_price_sale").text).strip())
except AttributeError:
    product_dict['price'] = int(re.sub(r'\([^)]*\)|,', '', soup.select_one("strong#span_product_price_text").text).strip())

#### product name

In [11]:
product_dict['name'] = soup.select_one("div.xans-element-.xans-product.xans-product-detaildesign > table > tbody > tr.xans-record- > td > span").text

#### product gender

In [12]:
product_dict['gender'] = "F"

#### product subcategory_id

In [13]:
product_dict['sub_category_id'] = 9

#### product category_id

In [14]:
product_dict['category_id'] = constants.SUB2CAT[product_dict['sub_category_id']]

#### product url

In [15]:
product_dict['url'] = product_url

#### product mall_id

In [16]:
product_dict['mall_id'] = constants.DAILYJOU_ID

#### product text

In [17]:
product_dict["description_path"] = "NULL"

#### product images

In [18]:
img_url_lst = list(map(lambda x: constants.DAILYJOU_ROOT_URL + x['ec-data-src'], soup.select_one('div.cont').select('img')))

#### product size

In [4]:
soup.find("iframe", id="snapfit_resultview_iframe")

In [15]:
size_url = "https:" + soup.select_one("iframe")['src']

In [21]:
soup.select("iframe")

[<iframe border="0" class="sf_chart_iframe_id_591937" frameborder="0" height="1305" id="sf_chart_iframe_id" scrolling="no" src="//cdn.snapfit.co.kr/stores/dailyjou/chart/591937.html" width="100%"> </iframe>]

In [9]:
size_url

'https://cdn.snapfit.co.kr/stores/dailyjou/chart/590440.html'

In [6]:
size_url

'https://cdn.snapfit.co.kr/stores/dailyjou/chart/590440.html'

In [21]:
response = requests.get(size_url, headers={"User-Agent": "Mozilla/5.0"})
html = response.content.decode('utf-8', 'replace')
soup = BeautifulSoup(html, "html.parser")

In [22]:
size_table = soup.select_one("table.sf_size_view.sfc_2baeb41-table.sfc_2baeb41-sf_size_view")

In [23]:
def table2df(table):
    data = []
    for row in table.find_all("tr"):
        row_data = [cell.get_text() for cell in row.find_all(["th", "td"])]
        data.append(row_data)

    df = pd.DataFrame(data)

    # # 첫 번째 행을 헤더로 설정
    new_header = df.iloc[0]
    df = df[1:]
    df.columns = new_header
    return df

In [24]:
size_df = table2df(size_table)

In [25]:
size_df

,사이즈,어깨,가슴,암홀,소매길이,소매통,총길이,밑단
1,FREE,52,57,24,65,21,69,51


In [26]:
size_dict_lst = []
for idx in size_df.index:
    row = size_df.loc[idx]
    size_dict = {
        'name': row['사이즈'],
        "product_id": "NULL",
        "top_id": "NULL",
        "outer_id": "NULL",
        "bottom_id": "NULL",
        "dress_id": "NULL",
    }

    cat_size_dict = {}
    for col, key in constants.DAILYJOU_TOP_SIZE_COL2KEY.items():
        if col not in row.index:
            cat_size_dict[key] = "NULL"
        else:
            cat_size_dict[key] = float(row[col])
    
    size_dict_lst += [[size_dict, cat_size_dict]]

In [27]:
size_dict_lst

[[{'name': 'FREE',
   'product_id': 'NULL',
   'top_id': 'NULL',
   'outer_id': 'NULL',
   'bottom_id': 'NULL',
   'dress_id': 'NULL'},
  {'full': 69.0, 'shoulder': 52.0, 'chest': 57.0, 'sleeve': 65.0}]]

### 최종 product_dict, size_dict_lst, img_url_lst, text

In [28]:
product_dict, size_dict_lst, img_url_lst

({'disabled': 'FALSE',
  'price': 34500,
  'name': '베베 빈티지 레터링 오버핏 후드 티셔츠 - 3 color',
  'gender': 'F',
  'sub_category_id': 9,
  'category_id': 2,
  'url': 'https://dailyjou.com/product/베베-빈티지-레터링-오버핏-후드-티셔츠-3-color/15355/category/46/display/1/',
  'mall_id': 2,
  'description_path': 'NULL'},
 [[{'name': 'FREE',
    'product_id': 'NULL',
    'top_id': 'NULL',
    'outer_id': 'NULL',
    'bottom_id': 'NULL',
    'dress_id': 'NULL'},
   {'full': 69.0, 'shoulder': 52.0, 'chest': 57.0, 'sleeve': 65.0}]],
 ['https://dailyjou.com/web/upload/NNEditor/20230901/0-1.jpg',
  'https://dailyjou.com/web/upload/NNEditor/20230901/1281-529.jpg',
  'https://dailyjou.com/web/upload/NNEditor/20230901/23_top_mj_00.jpg',
  'https://dailyjou.com/web/upload/NNEditor/20230901/23_top_mj_01.jpg',
  'https://dailyjou.com/web/upload/NNEditor/20230901/23_top_mj_02.jpg',
  'https://dailyjou.com/web/upload/NNEditor/20230901/23_top_mj_03.jpg',
  'https://dailyjou.com/web/upload/NNEditor/20230901/23_top_mj_04.jpg',
  '

## 바지 크롤링

In [2]:
subcategory_id = 12
product_url = "https://dailyjou.com/product/%EC%96%BC%EB%A6%AC%EB%B2%84%EB%93%9C207%EC%B2%9C%EC%9E%A5%ED%8C%90%EB%A7%A4made-%ED%86%A0%EB%B8%8C-%EC%9B%8C%EC%8B%B1-%EC%99%80%EC%9D%B4%EB%93%9C-%EB%8D%B0%EB%8B%98-%ED%8C%AC%EC%B8%A0-2-color/6269/category/148/display/1/"

In [3]:
response = requests.get(product_url, headers={"User-Agent": "Mozilla/5.0"})
html = response.text
soup = BeautifulSoup(html, "html.parser")

product_dict = {}

product_dict["disabled"] = "FALSE"
product_dict["price"] = search.get_price(soup)
product_dict["name"] = soup.select_one(
    "div.xans-element-.xans-product.xans-product-detaildesign > table > tbody > tr.xans-record- > td > span"
).text
product_dict["gender"] = "F"
product_dict["sub_category_id"] = subcategory_id
product_dict["category_id"] = constants.SUB2CAT[product_dict["sub_category_id"]]
product_dict["url"] = product_url
product_dict["mall_id"] = constants.DAILYJOU_ID
product_dict["description_path"] = "NULL"

img_url_lst = search.get_img_url_lst(soup)
size_dict_lst = search.get_size_dict_lst(soup, product_dict["category_id"])

In [4]:
size_url = "https:" + soup.select_one("iframe")["src"]

response = requests.get(size_url, headers={"User-Agent": "Mozilla/5.0"})
html = response.content.decode("utf-8", "replace")
soup = BeautifulSoup(html, "html.parser")

size_table = soup.select_one(
    "table.sf_size_view.sfc_2baeb41-table.sfc_2baeb41-sf_size_view"
)
size_df = utils.table2df(size_table)
size_dict_lst = utils.df2size_dict_lst(size_df, product_dict['category_id'])

In [5]:
size_df

,사이즈,허리,엉덩이,밑위,허벅지,밑단,총길이
1,S,31,51.5,32,30,23,102
2,M,33,53.5,32.5,30.5,23.5,104
3,L,35,55.5,33,31,24,106


In [6]:
size_dict_lst

[[{'name': 'S',
   'product_id': 'NULL',
   'top_id': 'NULL',
   'outer_id': 'NULL',
   'bottom_id': 'NULL',
   'dress_id': 'NULL'},
  {'full': 102.0,
   'waist': 31.0,
   'thigh': 30.0,
   'rise': 32.0,
   'bottom_width': 23.0,
   'hip_width': 51.5}],
 [{'name': 'M',
   'product_id': 'NULL',
   'top_id': 'NULL',
   'outer_id': 'NULL',
   'bottom_id': 'NULL',
   'dress_id': 'NULL'},
  {'full': 104.0,
   'waist': 33.0,
   'thigh': 30.5,
   'rise': 32.5,
   'bottom_width': 23.5,
   'hip_width': 53.5}],
 [{'name': 'L',
   'product_id': 'NULL',
   'top_id': 'NULL',
   'outer_id': 'NULL',
   'bottom_id': 'NULL',
   'dress_id': 'NULL'},
  {'full': 106.0,
   'waist': 35.0,
   'thigh': 31.0,
   'rise': 33.0,
   'bottom_width': 24.0,
   'hip_width': 55.5}]]

In [6]:
search.crawling_product(subcategory_id=12, product_url=product_url)

({'disabled': 'FALSE',
  'price': 30400,
  'name': '[얼리버드20%/7천장판매/MADE] 토브 워싱 와이드 데님 팬츠 - 2 color',
  'gender': 'F',
  'sub_category_id': 12,
  'category_id': 4,
  'url': 'https://dailyjou.com/product/%EC%96%BC%EB%A6%AC%EB%B2%84%EB%93%9C207%EC%B2%9C%EC%9E%A5%ED%8C%90%EB%A7%A4made-%ED%86%A0%EB%B8%8C-%EC%9B%8C%EC%8B%B1-%EC%99%80%EC%9D%B4%EB%93%9C-%EB%8D%B0%EB%8B%98-%ED%8C%AC%EC%B8%A0-2-color/6269/category/148/display/1/',
  'mall_id': 2,
  'description_path': 'NULL'},
 [[{'name': 'S',
    'product_id': 'NULL',
    'top_id': 'NULL',
    'outer_id': 'NULL',
    'bottom_id': 'NULL',
    'dress_id': 'NULL'},
   {'full': 102.0,
    'waist': 31.0,
    'thigh': 30.0,
    'rise': 32.0,
    'bottom_width': 23.0,
    'hip_width': 51.5}],
  [{'name': 'M',
    'product_id': 'NULL',
    'top_id': 'NULL',
    'outer_id': 'NULL',
    'bottom_id': 'NULL',
    'dress_id': 'NULL'},
   {'full': 104.0,
    'waist': 33.0,
    'thigh': 30.5,
    'rise': 32.5,
    'bottom_width': 23.5,
    'hip_width': 53.5}]

### RDS, S3에 저장

In [29]:
conn, cursor = rds.connect()
s3_obj = s3.connect()

#### product 정보 RDS에 저장

In [48]:
product_id = rds.insert_product(conn, cursor, product_dict)

In [30]:
product_id = 369

#### procuct size RDS 저장

In [31]:
for size_dict, cat_size_dict in size_dict_lst:
    cat_size_id = rds.insert_cat_size(conn, cursor, cat_size_dict, product_dict['category_id'])
    size_dict['product_id'] = product_id

    size_dict[constants.DAILYJOU_CAT_SIZE_ID[product_dict['category_id']]] = cat_size_id
    rds.insert_size(conn, cursor, size_dict)

#### product image list S3에 저장

In [31]:
thumbnail_img_url

'https://dailyjou.com/web/product/medium/202309/2e8c0daa550ac5c26e6ccbd3e0c58821.gif'

In [32]:
def s3_rds_image_(product_id, img_url, s3_obj, conn, cursor, thumbnail):
    fname = s3.upload_image(s3_obj, img_url)
    imagepath_dict = {"url": fname, "product_id": product_id, "thumbnail": thumbnail}
    rds.insert_imagepath(conn, cursor, imagepath_dict)

In [33]:
s3_rds_image_(product_id, thumbnail_img_url, s3_obj, conn, cursor, "TRUE")

In [34]:
for img_url in img_url_lst:
    s3_rds_image_(product_id, img_url, s3_obj, conn, cursor, "FALSE")

In [36]:
s3_access_key = pd.read_csv(paths.S3_ACCESS_KEY_PATH)
s3_obj = boto3.client(
    "s3",
    aws_access_key_id=s3_access_key["Access key ID"].values[0],
    aws_secret_access_key=s3_access_key["Secret access key"].values[0],
    region_name="ap-northeast-2",
)

In [9]:
from PIL import Image
response = requests.get(thumbnail_img_url, headers=constants.HEADER)
gif_image = response.content
s3_obj.put_object(
    Bucket="fittering-ml-test-bucket",
    Key="test.gif",
    Body=gif_image,
    ContentType="image/jpg",
)

{'ResponseMetadata': {'RequestId': 'R8QN0SBMPAXH0G1D',
  'HostId': 'P0YFnvAmqoLqSvDM3umATDtgH8K33qmVAJFCM4fPBICqM7gXKelzpqll5niY1EtfRhIqVVJk8hU=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'P0YFnvAmqoLqSvDM3umATDtgH8K33qmVAJFCM4fPBICqM7gXKelzpqll5niY1EtfRhIqVVJk8hU=',
   'x-amz-request-id': 'R8QN0SBMPAXH0G1D',
   'date': 'Tue, 05 Sep 2023 12:40:59 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"c3a93af0e1a1794055d81753a0c0c2bd"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"c3a93af0e1a1794055d81753a0c0c2bd"',
 'ServerSideEncryption': 'AES256'}

In [10]:
test_gif = Image.open(
    s3_obj.get_object(Bucket="fittering-ml-test-bucket", Key="test.gif")["Body"]
)

In [11]:
imgs = []
for frame_n in range(test_gif.n_frames):
    test_gif.seek(frame_n)
    imgs += [test_gif.convert("RGB")]

In [42]:
def size_check(p):
    try:
        if "SIZE" in p.select_one("u").text:
            return True
        elif "FITTING" in p.select_one("u").text:
            return True
        else:
            return False
    except:
        return False
def size_text(soup):
    p_lst = soup.select("div.cont > p")
    idxs = []
    for i, p in enumerate(p_lst):
        if size_check(p):
            idxs.append(i)
            if len(idxs) == 2:
                break
    print(idxs)
    return list(map(lambda x: x.text.strip(), p_lst[idxs[0]+1:idxs[1]]))

In [43]:
for product_li in products_li:
    product_url = "https://www.slowand.com/" + product_li.select_one("div.thumbnail > div.prdImg > a")["href"]
    print(product_url)
    thumbnail_img_url = product_li.select_one("div.thumbnail > div.prdImg > a > img")["src"]

    response = requests.get(product_url, headers={"User-Agent": "Mozilla/5.0"})
    html = response.text
    soup = BeautifulSoup(html, "html.parser")

    product_dict = {}
    product_dict['price'] = int(re.search(r'\d+', soup.select_one("strong#span_product_price_text").text.strip().replace(",", "")).group())
    product_dict['name'] = soup.select_one("div.infoArea > h2").text.strip()
    product_dict['gender'] = "F"
    product_dict['sub_category_id'] = 9
    product_dict['category_id'] = constants.SUB2CAT[product_dict['sub_category_id']]
    product_dict['url'] = product_url
    product_dict['mall_id'] = constants.SLOWAND_ID
    text = list(filter(lambda x: x.select_one("strong > span").text == '상품간략설명', product_li.select("div.description > ul.xans-element-.xans-product.xans-product-listitem.spec > li")))[0].select("span")[1].find_all(string=True)
    text = '\n'.join(list(map(lambda x: x.strip(), text)))

    print(size_text(soup))

https://www.slowand.com//product/어텀베스트-5무료배송-slowmade-니트라이크-여리핏-골지티셔츠-5-color-프리오더-510일-소요/7720/category/46/display/2/
[63, 66]
['어깨32/가슴34/암홀18/팔기장58/팔단면8.5/소매단면9/밑단36/총장56', '']
https://www.slowand.com//product/무료배송2만장돌파-slowmade-솔트-클린-반팔티셔츠-스탠다드핏루즈핏-7-color-반팔크롭티여름반팔여름티셔츠/7466/category/46/display/2/
[94, 100]
['스탠다드핏-어깨36.5/가슴46/암홀18.5', '팔기장17.5/소매단면15.5/밑단46.5/총장51', '루즈핏-어깨41/가슴52/암홀22.5', '팔기장23/소매단면18/밑단53/총장61.5', '']
https://www.slowand.com//product/가을베스트무료배송-slowmade-여리핏-소프트-골지티셔츠-7-color-프리오더-510일-소요/6801/category/46/display/2/
[64, 68]
['어깨34/가슴44/암홀19/팔기장57/팔단면12.5', '소매단면9.5/밑단44/총장53', '']
https://www.slowand.com//product/무료배송질좋은기본티-slowmade-프리미엄-오가닉코튼-무지티셔츠-11-color/4606/category/46/display/2/
[91, 93]
['어깨50/가슴52.5/암홀23/팔기장53.5/팔단면17.5/소매단면14/밑단52/총장62']
https://www.slowand.com//product/기획특가-slowmade-심플베이직-슬림무지티셔츠-6-color-프리오더-510일-소요/4467/category/46/display/2/
[94, 97]
['어깨36.5/가슴40/암홀20/팔기장61/팔단면11/밑단42/총장64', '']
https://www.slowand.com//product/무료배송1만장돌파-slowmade

In [42]:
pattern = r'(\d+\.\d+)\(최소\)\/(\d+)\(최대\)'

def replace_average(match):
    min_value = float(match.group(1))
    max_value = float(match.group(2))
    average = (min_value + max_value) / 2
    return str(average)

In [43]:
input_text = '43.5(최소)/49(최대)'
input_text = re.sub(pattern, replace_average, input_text)

In [39]:
(43.5 + 49) / 2

46.25

In [44]:
input_text

'46.25'